### ------------------------------------------------------------------------------------------------------------

In [ ]:
import os

import pandas as pd

if 'COLAB_GPU' in os.environ:
    !git clone https://github.com/impulsecorp/PickStocks.git
    !mv PickStocks/*.py .
    !mv PickStocks/data .
    !pip install -U -qq -r PickStocks/requirements.txt

In [ ]:
import warnings
warnings.filterwarnings("ignore")
import system
from system import *
# small hack to prevent Colab error
try:
    from datablock import *
except:
    from datablock import *
seed

In [ ]:
%pylab inline

### Setup

In [ ]:
data_timeperiod = 'D'
data = get_data('SPY', period=data_timeperiod, nrows=None)
data = procdata_lite(data)

In [ ]:
# for inspectiion
print(data.shape)
data.head()

In [ ]:
system.train_set_end = 0.6 # percentage point specifying the training set end point (1.0 means all data is training set)
system.val_set_end = 1.0   # percentage point specifying the validation set end point (1.0 means no test set)
system.balance_data = 1
system.scale_data = 1

### ------------------------------------------------------------------------------------------------------------

### Base classifier test

In [ ]:
# Train LogisticRegression classifier on train data
clf, scaler = train_classifier(LogisticRegression, data)

In [ ]:
# Test on val data
equity, pf, trades = qbacktest(clf, scaler, data)

In [ ]:
trades.head()

### ------------------------------------------------------------------------------------------------------------

### LSTM Ensemble Training

In [ ]:
# This is only to get the data shapes
X_train, y_train = get_clean_Xy_3d(data.iloc[0:int(data.shape[0] * system.train_set_end)], window_size=8)
X_train.shape, y_train.shape

In [ ]:
clf, scaler = train_classifier(RecurrentNetEnsemble, data, n_classifiers=10, time_window_size=X_train.shape[1], # trainer params
                               type='lstm', input_dim=X_train.shape[2], window_size=X_train.shape[1],
                               num_layers=1, dropout=0.75,
                               hidden_dim=8, n_epochs=15, # LSTM params
                              )

In [ ]:
# Test on val data
*_, trades = qbacktest(clf, scaler, data, window_size=X_train.shape[1])

In [ ]:
trades.head()

### ------------------------------------------------------------------------------------------------------------

### GRU Ensemble Training

In [ ]:
clf, scaler = train_classifier(RecurrentNetEnsemble, data, n_classifiers=10, time_window_size=X_train.shape[1], # trainer params
                               type='gru', input_dim=X_train.shape[2], window_size=X_train.shape[1],
                               num_layers=1, dropout=0.75,
                               hidden_dim=8, n_epochs=15,  # GRU params
                              )

In [ ]:
# Test on val data
*_, trades = qbacktest(clf, scaler, data, window_size=X_train.shape[1])

In [ ]:
trades.head()

### ------------------------------------------------------------------------------------------------------------

### RNN Ensemble Training

In [ ]:
clf, scaler = train_classifier(RecurrentNetEnsemble, data, n_classifiers=10, time_window_size=X_train.shape[1], # trainer params
                               type='rnn', input_dim=X_train.shape[2], window_size=X_train.shape[1],
                               num_layers=1, dropout=0.75,
                               hidden_dim=8, n_epochs=15,  # RNN params
                              )

In [ ]:
# Test on val data
*_, trades = qbacktest(clf, scaler, data, window_size=X_train.shape[1])

In [ ]:
trades.head()

### ------------------------------------------------------------------------------------------------------------